# Data Analysis Skills

Data cleaning skills using a .csv of geotechnical data from ‘EMerald’. 

EMerald’s internal data structures are built around the Swedish geotechnical format – we have an open source library called libsgfdata (installable via pip or by cloning the github repo and following the README.me).

You can see an excel sheet named SP_TRO-TEL_88.xlsx in the folder. We have also provided a spreadsheet and the corresponding parsed sgf file SP_TRO-TEL_94.sgf / .xlsx. As you can see this SGF file does not contain all the data from the excel sheet, but only the subset that we’re interested in.

The task is to extract a similar dataset, or as much data as you can out of SP_TRO-TEL_88.xlsx and write an SGF file using the method outlined in the libsgfdata documentation.

# Spare link to libsgfdata repo: 

https://github.com/emerald-geomodelling/libsgfdata with documentation on how to write an SGF file

In [1]:
import libsgfdata

In [3]:
sgf_file = libsgfdata.SGFData("SP_TRO-TEL_94.sgf")

In [4]:
sgf_file.main

,CONTRATANTE,OBSERVAÇÃO,LOCAL,CIDADE,FISCAL,INÍCIO LAVAGEM,COMP. REVESTIMENTO,Título,method_code,x_coordinate,y_coordinate,work_or_project_number,section,investigation_point,date,responsible_drilling_operator_or_measurement_operator,project_name
0,Subcontratada Enefer,"Profundidade de projeto 15m,;Não atingiu a pro...",MT - MATO GROSSO,Rondonópolis,Ediemes Pereira Neves,2.45,"1,50",Perfil de Sondagem SPT,standard_penetration_test,"E 737.711,456","N 8.200.240,215",SP_TRO-TEL_94,SP_TRO-TEL_94,SP - 94,2020-11-17,Luiz Carlos da Conceição,Projeto Básico - Ferrovia Lucas do Rio Verde -...


In [5]:
sgf_file.data

,depth,test_no,ramming_nn_value,ramming_nn_1_value,comments,end_depth,start_depth,investigation_point
0,-1.00,0.0,0,0,Areia siltosa cinza,-1.00,0.00,SP - 94
1,-1.45,1.0,4,4,Areia siltosa cinza,-1.45,-1.00,SP - 94
2,-2.45,2.0,7,8,Areia siltosa cinza,-2.45,-1.45,SP - 94
3,-3.45,3.0,8,8,Areia siltosa cinza,-3.45,-2.45,SP - 94
4,-4.45,4.0,12,14,Areia siltosa cinza;v,-4.45,-3.45,SP - 94
5,-5.45,5.0,16,17,Areia siltosa cinza,-5.45,-4.45,SP - 94
6,-6.45,6.0,12,11,Areia siltosa cinza,-6.45,-5.45,SP - 94
7,-7.45,7.0,13,13,Areia siltosa cinza,-7.45,-6.45,SP - 94
8,-8.45,8.0,17,18,Areia siltosa cinza,-8.45,-7.45,SP - 94
9,-9.45,9.0,21,24,Areia siltosa cinza,-9.45,-8.45,SP - 94


Please put all working code below here

# ---------------------------------------

## Code to write an SGF file in ‘EMerald standard’
### Date: 2022.11.03
### Author: Akpo Siemuri

### Install required libraries

In [6]:
# Install required libraries
#!pip install libsgfdata
#!pip install openpyxl

### Import important modules

In [7]:
# import important modules
import openpyxl
from openpyxl import load_workbook
import libsgfdata as sgf
import pandas as pd
import numpy as np
import os
import datetime
import re
import libsgfdata # SGF file

## Transform Excel sheet

In [8]:
# Declear files here
excel_file = "SP_TRO-TEL_88.xlsx"
transformed = "tranformed88.txt"

# Read Excel sheet
excel = pd.read_excel(excel_file, sheet_name="Sondagem", engine='openpyxl')
with open(transformed, 'w') as outfile:
    excel.to_string(outfile)
    # remove cell with whitespaces
    excel = excel.replace({"                                                                                                                  ":np.nan})

# exclude columns you don't want
excel[excel.columns[~excel.columns.isin(['Unnamed: 10','Unnamed: 11'])]]
excel = excel.drop(excel.iloc[:, 19:39], axis=1)

# Useful columns
excel = excel.iloc[:, 0:18]
# create two important columns from selected rows of column:'Unnamed: 0'
excel['end_depth'] = excel.loc[12:26, 'Unnamed: 0']
excel['start_depth'] = excel.loc[12:24, 'Unnamed: 0'].shift().fillna(0)

# write to transformed excel file
excel.to_excel('NEW_' + excel_file)
new_excel_file = 'NEW_' + excel_file
print(new_excel_file)
print('Finnished processing Excel transform!')

NEW_SP_TRO-TEL_88.xlsx
Finnished processing Excel transform!


## Extract heading for SGF file

In [9]:
# Declear files here
append_to_file = "sp_tro_tel_88.txt"

# Extract heading for SGF file

today = datetime.date.today() # for converting datetime into string
# using 'today' will check only the date (not time)
def iterating_over_values(path, sheet_name, cell_range):
    workbook = load_workbook(filename=path)
    # Check for worksheet names (tabs)
    if sheet_name not in workbook.sheetnames:
        print(f"'{sheet_name}' not found. Quitting.")
        return
    # Select the worksheet to parse
    if sheet_name == workbook.sheetnames[0]:
        sheet = workbook[sheet_name]
        # Iterate via selected worksheet to get data from specified cell range
        for row in sheet[cell_range]:
            for cell in row:
                # convert datetime into string
                if cell.is_date and cell.value.date() != today:
                    cell.value = str(cell.value).rstrip(' 00:00:00')
                if cell.value != None:
                    print(f"{cell.column_letter}{cell.row} = {cell.value}")
                    # Append values to file
                    with open(append_to_file, 'a') as file:
                        file.writelines(str(cell.value))
                        if str(cell.value).find(':') == -1:
                            file.writelines(",")
                        if str(cell.value).find('19m') != -1 or str(cell.value).find('15m') != -1 or str(cell.value).find('tada.') != -1:
                            file.writelines(";")                        

if __name__ == "__main__":
    
    # Open file and write the initial 'header block $' sign
    with open(append_to_file, 'w') as file:
        file.writelines("$\n")
        
    # parse selected data to text file
    iterating_over_values(new_excel_file, sheet_name="Sheet1", cell_range="B4:R6")
    iterating_over_values(new_excel_file, sheet_name="Sheet1", cell_range="C44:C53")
    iterating_over_values(new_excel_file, sheet_name="Sheet1", cell_range="Q39:S52")
    #iterating_over_values(new_excel_file, sheet_name="Sheet1", cell_range="J52:J53")
    
    # Rename some key libsgfdata columns like HK:'investigation_point'
    with open(append_to_file) as f:
        header = f.read()
        string0 = header.replace("FURO", ",HK")
        string1 = string0.replace("COORDENADAS", "HX")
        string2 = string1.replace("N ", "HY=N ")
        string3 = string2.replace("DATA DO ,HK", "KD")
        string4 = string3.replace(":", "=")
        string5 = string4.replace("OPERADOR", "Person")
        string6 = string5.replace("EMPRESA DE FISCALIZAÇÃO=FISCAL= ", "FISCAL=")
        string7 = string6.replace("OBRA", "KP")
        string8 = string7.replace(" Perfil de Sondagem  SPTPLANO DE SONDAGENS", "Perfil de Sondagem  SPT")
        string9 = string8.replace("Número=  ", "HJ=")
        string10 = string9.replace("TRECHO= ", ",HL=")
        Mheader = string10.replace(",;HY", ",")
        print(Mheader)

    # write final data to text file in SGF format
    with open(excel_file.rstrip('xlsx') + 'sgf', 'w') as final:
        final.writelines(Mheader)
    print('Finnished processing the header of SGF file!')

B4 = Título: Perfil de Sondagem  SPT
Q4 = PLANO DE SONDAGENS
Q5 = Número:  SP_TRO-TEL_88
B6 = TRECHO: SP_TRO-TEL_88
Q6 = FURO:
R6 = SP - 88
C44 = CONTRATANTE:
C45 = Subcontratada Enefer
C46 = OBRA:
C47 = Projeto Básico - Ferrovia Lucas do Rio Verde - Trecho TRO-TEL
C48 = LOCAL: 
C49 = MT - MATO GROSSO
C50 = CIDADE:
C51 = Rondonópolis
C52 = INÍCIO LAVAGEM:
C53 = 2020-11-12
Q40 = COORDENADAS:
R40 = E 739.017,126
R41 = N 8.195.326,187
Q44 = OBSERVAÇÃO:
R44 = Profundidade de projeto 19m
R45 = Não atingiu a profundidade projetada.
R46 = Início da lavagem 2,45
Q47 = DATA DO FURO:
R47 = 2020-11-12
Q49 = EMPRESA DE FISCALIZAÇÃO:
Q50 = FISCAL: 
R50 = Ediemes Pereira Neves
Q51 = OPERADOR: 
R51 = Rodrigo Borges Leonel
Q52 = RESPONSÁVEL TÉCNICO:
S52 = José Borges Junior
$
Título=Perfil de Sondagem  SPT,HJ=SP_TRO-TEL_88,HL=SP_TRO-TEL_88,HK=SP - 88,CONTRATANTE=Subcontratada Enefer,KP=Projeto Básico - Ferrovia Lucas do Rio Verde - Trecho TRO-TEL,LOCAL= MT - MATO GROSSO,CIDADE=Rondonópolis,INÍCIO LAVA

## Extract Body for SGF file

In [10]:
# Extract Body for SGF file

def iterating_over_values(path, sheet_name, cell_range):
    workbook = load_workbook(filename=path)
    # Check for worksheet names (tabs)
    if sheet_name not in workbook.sheetnames:
        print(f"'{sheet_name}' not found. Quitting.")
        return
    # Select the worksheet to parse
    if sheet_name == workbook.sheetnames[0]:
        sheet = workbook[sheet_name]
        # Iterate via selected worksheet to get data from specified cell range
        for row in sheet[cell_range]:
            for cell in row:
                if cell.value != None:
                    print(f"{cell.column_letter}{cell.row} = {cell.value}")
                    # write to SGF based on columns of interest
                    with open(excel_file.rstrip('xlsx') + 'sgf', 'a') as file:
                        if str(cell.column_letter).startswith('B'):
                            file.writelines('D=' + str(cell.value))
                            file.writelines(",")
                        elif str(cell.column_letter).startswith('F'):
                            file.writelines('ProvNr=' + str(cell.value))
                            file.writelines(",")
                        elif str(cell.column_letter).startswith('H'):
                            file.writelines('Nn=' + str(cell.value))
                            file.writelines(",")
                        elif str(cell.column_letter).startswith('I'):
                            file.writelines('Nn+1=' + str(cell.value))
                            file.writelines(",")
                        elif str(cell.column_letter).startswith('P'):
                            file.writelines('K=' + str(cell.value))
                            file.writelines(",")
                        elif str(cell.column_letter).startswith('T'):
                            file.writelines('AO=' + str(cell.value))
                            file.writelines(",")
                        elif str(cell.column_letter).startswith('U'):
                            file.writelines('AN=' + str(cell.value))
                            file.writelines("\n")
                            file.close()
                            
if __name__ == "__main__":
    
    # Open file ('SP_TRO-TEL_XX.sgf') and write 'body block #' sign
    body_marker = '#'
    with open(excel_file.rstrip('xlsx') + 'sgf', 'r') as file:
        # read all content of a file
        content = file.read()
        # check if string (body_marker) is present in the file
        if body_marker in content:
            print('SGF body already exist. Quitting.')
        else:
            print('SGF body does not exist. Writing SGF body.') 
            with open(excel_file.rstrip('xlsx') + 'sgf', 'a') as file:
                file.writelines("\n#\n")
        
            # parse selected data to text file
            iterating_over_values(new_excel_file, sheet_name="Sheet1", cell_range="B14:U26")
            print('Finnished processing the body of SGF file!')

SGF body does not exist. Writing SGF body.
B14 = -1
F14 = 0
H14 = 0
I14 = 0
P14 = Areia siltosa amarela
T14 = -1
U14 = 0
B15 = -1.45
F15 = 1
H15 = 4
I15 = 4
P15 = Areia siltosa amarela
T15 = -1.45
U15 = -1
B16 = -2.45
F16 = 2
H16 = 4
I16 = 3
P16 = Areia siltosa amarela
T16 = -2.45
U16 = -1.45
B17 = -3.45
F17 = 3
H17 = 4
I17 = 5
P17 = Areia siltosa amarela
T17 = -3.45
U17 = -2.45
B18 = -4.45
F18 = 4
H18 = 6
I18 = 7
P18 = Areia amarela
T18 = -4.45
U18 = -3.45
B19 = -5.45
F19 = 5
H19 = 6
I19 = 5
P19 = Areia amarela
T19 = -5.45
U19 = -4.45
B20 = -6.45
F20 = 6
H20 = 10
I20 = 11
P20 = Areia amarela
T20 = -6.45
U20 = -5.45
B21 = -7.45
F21 = 7
H21 = 10
I21 = 11
P21 = Areia amarela
T21 = -7.45
U21 = -6.45
B22 = -8.45
F22 = 8
H22 = 13
I22 = 15
P22 = Areia amarela
T22 = -8.45
U22 = -7.45
B23 = -9.45
F23 = 9
H23 = 16
I23 = 15
P23 = Areia amarela
T23 = -9.45
U23 = -8.45
B24 = -10.45
F24 = 10
H24 = 27
I24 = 27
P24 = Areia amarela
T24 = -10.45
U24 = -9.45
B25 = -11.45
F25 = 11
H25 = 29
I25 = 32
P25 =

## Process the SGF file

In [11]:
new_sgf_file = libsgfdata.SGFData(excel_file.rstrip('xlsx') + 'sgf')

In [12]:
new_sgf_file.main

,Título,work_or_project_number,section,investigation_point,CONTRATANTE,project_name,LOCAL,CIDADE,INÍCIO LAVAGEM,x_coordinate,y_coordinate,OBSERVAÇÃO,date,FISCAL,responsible_drilling_operator_or_measurement_operator,RESPONSÁVEL TÉCNICO
0,Perfil de Sondagem SPT,SP_TRO-TEL_88,SP_TRO-TEL_88,SP - 88,Subcontratada Enefer,Projeto Básico - Ferrovia Lucas do Rio Verde -...,MT - MATO GROSSO,Rondonópolis,2020-11-12,"E 739.017,126","N 8.195.326,187","Profundidade de projeto 19m,;Não atingiu a pro...",2020-11-12,Ediemes Pereira Neves,Rodrigo Borges Leonel,José Borges Junior


In [13]:
new_sgf_file.data

,depth,test_no,ramming_nn_value,ramming_nn_1_value,comments,end_depth,start_depth,investigation_point
0,-1.00,0,0,0,Areia siltosa amarela,-1.00,0.00,SP - 88
1,-1.45,1,4,4,Areia siltosa amarela,-1.45,-1.00,SP - 88
2,-2.45,2,4,3,Areia siltosa amarela,-2.45,-1.45,SP - 88
3,-3.45,3,4,5,Areia siltosa amarela,-3.45,-2.45,SP - 88
4,-4.45,4,6,7,Areia amarela,-4.45,-3.45,SP - 88
5,-5.45,5,6,5,Areia amarela,-5.45,-4.45,SP - 88
6,-6.45,6,10,11,Areia amarela,-6.45,-5.45,SP - 88
7,-7.45,7,10,11,Areia amarela,-7.45,-6.45,SP - 88
8,-8.45,8,13,15,Areia amarela,-8.45,-7.45,SP - 88
9,-9.45,9,16,15,Areia amarela,-9.45,-8.45,SP - 88
